### Library

In [10]:
import pandas as pd
import numpy as np
from selenium import webdriver as wb 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By 
import time
from tqdm import tqdm
import re
import glob # to get all files in a directory

### Crawling

In [2]:
driver = wb.Chrome()
url = 'https://www.cbtbank.kr/category/정보처리기사'
driver.get(url)
time.sleep(5)

In [3]:
index = 1
exam_tags = driver.find_elements(By.CSS_SELECTOR, 'div.col.p-2.text-primary > a')
date = re.search(r"\d{4}-\d{2}-\d{2}", exam_tags[index].text).group()
exam_tags[index].click()

In [4]:
date

'2020-08-22'

In [5]:
pb_num, pb_ques, pb_detail = [], [], []

for tag in driver.find_elements(By.CSS_SELECTOR, 'p.exam-title'):
    parts = tag.text.split('.')
    pb_num.append(int(parts[0]))
    pb_ques.append('.'.join(parts[1:]).strip())

    try:
        img_tag = tag.find_element(By.TAG_NAME, 'img')
        pb_detail.append(img_tag.get_attribute('src'))
    except:
        pb_detail.append(np.nan)

In [6]:
pb_choi1, pb_choi2, pb_choi3, pb_choi4, pb_ans = [], [], [], [], []

for tag in driver.find_elements(By.CSS_SELECTOR, 'ol.circlednumbers'):
    try: 
        pb_choi1_tmp = tag.text.split('\n')[0]
        pb_choi2_tmp = tag.text.split('\n')[1]
        pb_choi3_tmp = tag.text.split('\n')[2]
        pb_choi4_tmp = tag.text.split('\n')[3]
    except:
        choi_img_tags = tag.find_elements(By.TAG_NAME, 'img')
        pb_choi1_tmp = choi_img_tags[0].get_attribute('src')
        pb_choi2_tmp = choi_img_tags[1].get_attribute('src')
        pb_choi3_tmp = choi_img_tags[2].get_attribute('src')
        pb_choi4_tmp = choi_img_tags[3].get_attribute('src')

    pb_choi1.append(pb_choi1_tmp)
    pb_choi2.append(pb_choi2_tmp)
    pb_choi3.append(pb_choi3_tmp)
    pb_choi4.append(pb_choi4_tmp)
    pb_ans.append(int(tag.get_attribute('correct')))

In [7]:
pb_cr = []
for tag in driver.find_elements(By.CSS_SELECTOR, 'span.exam-cpercent'):
    pb_cr.append(int(re.search(r"\d{2}", tag.text).group()))

In [8]:
len(pb_num), len(pb_ques), len(pb_detail), len(pb_choi1), len(pb_choi2), len(pb_choi3), len(pb_choi4), len(pb_ans), len(pb_cr)

(100, 100, 100, 100, 100, 100, 100, 100, 100)

In [9]:
# pb_ques 에 ? 없는 제목 찾기
for i in range(len(pb_ques)):
    if '?' not in pb_ques[i]:
        print(i, pb_ques[i])

In [ ]:
pd.DataFrame({
    'PB_ID' : [np.nan] * 100,
    'LI_ID' : [2] * 100, # 정보처리기사 자격증 번호 1번, 산업안전산업기사 2번
    'EX_ID' : [8-index+1] * 100, # 시험 회차 데이터
    'PB_NUM': pb_num,   
    'PB_QUES': pb_ques,
    'PB_DETAIL': pb_detail,
    'PB_CHOI1': pb_choi1,
    'PB_CHOI2': pb_choi2,
    'PB_CHOI3': pb_choi3,
    'PB_CHOI4': pb_choi4,
    'PB_ANS': pb_ans,
    'PB_SOLU' : [np.nan] * 100,
    'PB_CR': pb_cr,
    'PB_TOPIC' : [np.nan] * 100
}).to_csv(f'../Data/정보처리기사_{date}.csv', index=False, encoding='utf-8')

driver.back()

### 데이터 병합

In [12]:
file_list = glob.glob("../Data/*정보처리기사*")
df_list = [pd.read_csv(f) for f in file_list]
merged_df = pd.concat(df_list, ignore_index=True)
merged_df['PB_ID'] = range(1, len(merged_df) + 1)
merged_df

,PB_ID,EX_ID,PB_NUM,PB_QUES,PB_DETAIL,PB_CHOI1,PB_CHOI2,PB_CHOI3,PB_CHOI4,PB_ANS,PB_SOLU,PB_CR,PB_TOPIC
0,1,1,1,검토회의 전에 요구사항 명세서를 미리 배포하여 사전 검토한 후 짧은 검토 회의를 통...,NaN,빌드 검증,동료 검토,워크 스루,개발자 검토,3,NaN,82,NaN
1,2,1,2,코드 설계에서 일정한 일련번호를 부여하는 방식의 코드는?,NaN,연상 코드,블록 코드,순차 코드,표의 숫자 코드,3,NaN,85,NaN
2,3,1,3,객체지향 프로그램에서 데이터를 추상화하는 단위는?,NaN,메소드,클래스,상속성,메시지,2,NaN,83,NaN
3,4,1,4,데이터 흐름도(DFD)의 구성요소에 포함되지 않는 것은?,NaN,process,data flow,data store,data dictionary,4,NaN,83,NaN
4,5,1,5,소프트웨어 설계시 구축된 플랫폼의 성능특성 분석에 사용되는 측정 항목이 아닌 것은?,NaN,응답시간(Response Time),가용성(Availability),사용률(Utilization),서버 튜닝(Server Tuning),4,NaN,90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,8,96,COCOMO(Constructive Cost Model) 모형의 특징이 아닌 것은?,NaN,프로젝트를 완성하는데 필요한 man-month로 산정 결과를 나타낼 수 있다.,보헴(Boehm)이 제안한 것으로 원시코드 라인 수에 의한 비용 산정 기법이다.,비교적 작은 규모의 프로젝트 기록을 통계 분석하여 얻은 결과를 반영한 모델이며 중소...,"프로젝트 개발유형에 따라 object, dynamic, function의 3가지 모...",4,NaN,34,NaN
796,797,8,97,각 사용자 인증의 유형에 대한 설명으로 가장 적절하지 않은 것은?,NaN,"지식 : 주체는 '그가 알고 있는 것'을 보여주며 예시로는 패스워드, PIN 등이 있다.","소유 : 주체는 '그가 가지고 있는 것'을 보여주며 예시로는 토큰, 스마트카드 등이...","존재 : 주체는 '그를 대체하는 것'을 보여주며 예시로는 패턴, QR 등이 있다.","행위 : 주체는 '그가 하는 것'을 보여주며 예시로는 서명, 움직임, 음성 등이 있다.",3,NaN,52,NaN
797,798,8,98,시스템의 사용자가 로그인하여 명령을 내리는 과정에 대한 시스템의 동작 중 다음 설명...,https://www.cbtbank.kr/images/iz/iz20220424/iz...,Aging,Accounting,Authorization,Authentication,4,NaN,31,NaN
798,799,8,99,다음에서 설명하는 IT 기술은?,https://www.cbtbank.kr/images/iz/iz20220424/iz...,SDN(Software Defined Networking),NFS(Network File System),Network Mapper,AOE Network,1,NaN,37,NaN


In [14]:
merged_df.isnull().sum() # 결측치 확인

PB_ID          0
EX_ID          0
PB_NUM         0
PB_QUES        0
PB_DETAIL    632
PB_CHOI1       0
PB_CHOI2       0
PB_CHOI3       0
PB_CHOI4       0
PB_ANS         0
PB_SOLU      800
PB_CR          0
PB_TOPIC     800
dtype: int64

In [15]:
merged_df.to_csv('../Data/EIP_Problem_All.csv', index=False, encoding='utf-8')